In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'mayfly/data/datasets')


def linear_fit(x, a, b):
    
    return a + b * x


In [ ]:
scores = np.load(os.path.join(RESULTPATH,'210810_mf_scores_84_100.npy'))
scores_2 = np.load(os.path.join(RESULTPATH,'210810_mf_scores_84_100_2.npy'))
print(scores.shape)

In [ ]:
test_dataset = '210810_84_100_rng_test.h5'
template_dataset = '210810_84_100_templates.h5'
template_dataset_2 = '210810_84_100_templates.h5'

mftestdata = mf.data.MFDataset(os.path.join(DATAPATH, test_dataset))
mftemplates = mf.data.MFDataset(os.path.join(DATAPATH, template_dataset))
mftemplates_2 = mf.data.MFDataset(os.path.join(DATAPATH, template_dataset))

param_list = ['theta_min', 'energy']

test_metadata = pd.DataFrame(mftestdata.metadata)[param_list]
template_metadata = pd.DataFrame(mftemplates.metadata)[param_list]
template_metadata_2 = pd.DataFrame(mftemplates_2.metadata)[param_list]

var = 1.38e-23 * 10 * 50 * 200e6
ideal_scores = abs((1 / np.sqrt(var * np.sum(mftestdata.data[:] * mftestdata.data[:].conjugate(), axis=-1))) * np.sum(mftestdata.data[:] * mftestdata.data[:].conjugate(), axis=-1))

In [ ]:
mftestdata.data

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)

hist = ax.hist(test_metadata['energy'], 64)

ax.set_ylabel('N')
ax.set_xlabel(r'$E$ (eV)')
ax.set_title('Signal Energies')

plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, '210811_mf_84_100_test_energy_dist.png'))

In [ ]:
match = np.array([scores.max(axis=0), scores_2.max(axis=0)]).max(axis=0) / ideal_scores
print(np.mean(match))

sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(8,5))

ax = fig.add_subplot(1,1,1)

hist = ax.hist(ideal_scores, 64, histtype='step', label='Ideal Scores')
hist = ax.hist(np.array([scores.max(axis=0), scores_2.max(axis=0)]).max(axis=0), 64, histtype='step', label='Template Scores')

ax.legend(loc=1)


ax.set_title(r'Distribution of Ideal Scores vs Best Template Scores, $N=8192$')
ax.set_xlabel('Score')
ax.set_ylabel(r'$N$')

plt.savefig(os.path.join(PLOTPATH, '210816_ideal_scores_vs_template_scores.png'))

In [ ]:
np.argwhere(ideal_scores >= 2).squeeze().size / len(ideal_scores)

In [ ]:
np.argwhere(np.array([scores.max(axis=0), scores_2.max(axis=0)]).max(axis=0) >=2).squeeze().size / len(ideal_scores)

In [ ]:
.255 / .53

# plot signal and template distributions

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

pitch_angles = np.linspace(84, 90, 61)
energies = np.linspace(18500, 18600, 401)

angle_grid, energy_grid = np.meshgrid(pitch_angles, energies)

line1 = ax.plot(energy_grid.flatten(), angle_grid.flatten(), '.', markersize=10, label='Template')

signal_e = test_metadata['energy'].array
signal_angle = test_metadata['theta_min'].array
#print(signal_angle)

line2 = ax.plot(np.array(signal_e), np.array(signal_angle), '.', markersize=10, label='Signal')

#plt.xlim(18500, 18600)
#plt.ylim(84, 90)

ax.set_ylabel(r'$\theta$ (deg)')
ax.set_xlabel(r'$E$ (eV)')
ax.set_title('Distribution of Template and Test Signal Parameters')
ax.legend(loc=(0.97, 0.90))

plt.tight_layout()

#ax.set_xlim(18500, 18520)
#ax.set_ylim(89, 90)
#plt.savefig(os.path.join(PLOTPATH, '210811_signal_and_template_parameter_distributions_zoom.png'))

In [ ]:
pitch_angles = np.linspace(84, 90, 61)
energies = np.linspace(18500, 18600, 201)
print(pitch_angles, energies)
template_metadata[(template_metadata['theta_min'].isin(pitch_angles)) & (template_metadata['energy'].isin(energies))]
inds = template_metadata[(template_metadata['theta_min'].isin(pitch_angles)) & (template_metadata['energy'].isin(energies))].index.array

# vary pitch angles used as templates and compute mean match

In [ ]:
pitch_angles_all = np.linspace(84, 90, 61)
energies_all = np.linspace(18500, 18600, 401)

mean_match = np.zeros(pitch_angles_all.size)
for n_pitch_angles in range(pitch_angles_all.size):
    
    pitch_angle_inds = np.int32(np.linspace(0, 60, n_pitch_angles+1))
    
    selected_angles = pitch_angles_all[pitch_angle_inds]
    
    inds = template_metadata[(template_metadata['theta_min'].isin(selected_angles)) & (template_metadata['energy'].isin(energies_all))].index.array
    
    if len(inds) == 0:
        inds = template_metadata_2[(template_metadata_2['theta_min'].isin(selected_angles)) & (template_metadata_2['energy'].isin(energies_all))].index.array
    
        mean_match[n_pitch_angles] = np.mean(scores_2[inds, :].max(axis=0) / ideal_scores)


In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(16,7))
ax = fig.add_subplot(1,2,1)

#popt, pcov = scipy.optimize.curve_fit(linear_fit, np.log10(np.arange(1, 62, 1)), mean_match)

ax.plot(np.arange(1, 62, 1), mean_match)
ax.set_title('Linear Scale')
ax.set_ylabel('Mean Match')
ax.set_xlabel('N')
ax2 = fig.add_subplot(1,2,2)

ax2.plot(np.arange(1, 62, 1), mean_match)
ax2.set_title('Log Scale')
ax2.set_xscale('log')
ax2.set_xlabel('N')
#ax.plot(np.log10(np.arange(1, 62, 1)), linear_fit(np.log10(np.arange(1, 62, 1)), *popt))
#ax.plot()
#plt.xscale('log')

fig.suptitle('Mean Match vs Pitch Angles in Template Bank')
plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, '210811_match_vs_pitch_angle.png'))

# vary energies used as templates and compute mean match

In [ ]:
pitch_angles_all = np.linspace(84, 90, 61)
energies_all = np.linspace(18500, 18600, 201)

mean_match = np.zeros(energies_all.size)
for n_energies in range(energies_all.size):
    
    energy_inds = np.int32(np.linspace(0, 200, n_energies+1))
    #print(energy_inds)
    selected_energies = energies_all[energy_inds]
    
    inds = template_metadata[(template_metadata['theta_min'].isin(pitch_angles_all)) & (template_metadata['energy'].isin(selected_energies))].index.array
    #print(scores[inds, :].max(axis=0))
    mean_match[n_energies] = np.mean(scores[inds, :].max(axis=0) / ideal_scores)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(16,7))
ax = fig.add_subplot(1,2,1)

#popt, pcov = scipy.optimize.curve_fit(linear_fit, np.log10(np.arange(1, 62, 1)), mean_match)

ax.plot(np.arange(1, 202, 1), mean_match)
ax.set_title('Linear Scale')
ax.set_ylabel('Mean Match')
ax.set_xlabel('N')
ax2 = fig.add_subplot(1,2,2)

ax2.plot(np.arange(1, 202, 1), mean_match)
ax2.set_title('Log Scale')
ax2.set_xscale('log')
ax2.set_xlabel('N')
#ax.plot(np.log10(np.arange(1, 62, 1)), linear_fit(np.log10(np.arange(1, 62, 1)), *popt))
#ax.plot()
#plt.xscale('log')

fig.suptitle('Mean Match vs Energies in Template Bank')
plt.tight_layout()
#plt.savefig(os.path.join(PLOTPATH, '210811_match_vs_energy.png'))

# density of parameters

In [ ]:
area = 100 * 6 # eV * degrees

In [ ]:
density = 12261 / area

In [ ]:
print(f'Template density = {density} templates / (eV*deg)')

# vary the density of parameters and plot the mean match

In [ ]:
pitch_angles_all = np.linspace(84, 90, 61)
energies_all = np.linspace(18500, 18600, 401)

N_densities = 61

N_pitch_angles = np.int32(np.linspace(1, 61, N_densities))
N_energies = np.int32(np.linspace(1, 201, N_densities))

mean_match = np.zeros(N_densities)
for i in range(N_densities):
    
    energy_inds = np.int32(np.linspace(0, 200, N_energies[i]+1))
    angle_inds = np.int32(np.linspace(0, 60, N_pitch_angles[i]+1))

    selected_energies = energies_all[energy_inds]
    selected_angles = pitch_angles_all[angle_inds]
    
    inds = template_metadata[(template_metadata['theta_min'].isin(selected_angles)) & (template_metadata['energy'].isin(selected_energies))].index.array
    if len(inds) == 0:
        inds = template_metadata_2[(template_metadata_2['theta_min'].isin(selected_angles)) & (template_metadata_2['energy'].isin(selected_energies))].index.array
    #print(scores[inds, :].max(axis=0))
        mean_match[i] = np.mean(scores_2[inds, :].max(axis=0) / ideal_scores)
    else:
        mean_match[i] = np.mean(scores[inds, :].max(axis=0) / ideal_scores)

In [ ]:
sns.set_theme(context='talk', style='ticks')

fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(1,1,1)

def density2number(x):
    return 600 * x
def number2density(x):
    return x / 600

ax.plot(density, mean_match)
ax.set_xscale('log')
ax.set_xlabel(r'Template Density $\left(\frac{N}{eV\cdot deg}\right)$')
ax.set_ylabel('Mean Match')

ax2 = ax.secondary_xaxis(-0.3, functions=(density2number, number2density))
ax2.set_xlabel(r'$N$')
ax.grid()

ax.set_title('Mean Match vs Template Density')

plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210816_mean_match_vs_template_density_and_number.png'))


In [ ]:
density = N_pitch_angles * N_energies / (6 * 100)


# plot histogram of match ratios for different pitch angle ranges

In [ ]:
print(scores.shape)
print(ideal_scores.shape)

In [ ]:
inds = test_metadata[(test_metadata['theta_min']<= 85.0) & (test_metadata['theta_min'] >= 84.0)].index.array

In [ ]:
angles = np.linspace(89, 84, 6)

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1,1,1)

for angle in angles:
    inds = test_metadata[(test_metadata['theta_min']<= angle+1) & (test_metadata['theta_min'] >= angle)].index.array
    hist = ax.hist(abs(scores[:, inds].max(axis=0) / ideal_scores[inds]), 32, histtype='step', label=f'{int(angle)}-{int(angle+1)}')
    
ax.legend(loc=(1.01, 0), title='Pitch Angles')
ax.set_xlabel('Match Ratio')
ax.set_ylabel('N')
ax.set_title('Match Ratio Pitch Angle Demographics')
plt.tight_layout()
plt.savefig(os.path.join(PLOTPATH, '210812_distributions_of_match_ratio_vs_pitch_angle.png'))

In [ ]:
np.linspace(84, 89, 6)